## styletest

输入图片，寻找和它同名的提示词，然后用图生图+提示词拼接，生成图片

In [1]:
import json
from urllib import request, parse
import random
import uuid
import urllib.request
import urllib.parse
import io
import os
import time
from PIL.PngImagePlugin import PngInfo
import hashlib
from urllib.error import URLError


In [2]:
def changeImageSize(width, height):
    # 最小的宽高为 896
    limited = 896
    minSize = min(width, height)

    # 缩放宽高，使得最窄的边等于limited
    if minSize > limited:
        scale = minSize / limited
        widthScale = int(width / scale)
        heightScale = int(height / scale)
    else:
        scale = limited / minSize
        widthScale = int(width * scale)
        heightScale = int(height * scale)

    # 将两个值都除以64，然后取整,也就是说边长必须是64的倍数
    widthScale = int(widthScale/64) * 64
    heightScale = int(heightScale/64) * 64

    return widthScale, heightScale

def checkImageExsit(imageDir, imageName):
    for file in os.listdir(imageDir):
        if file.startswith(imageName):
            return True
    return False


def queue_prompt(prompt):
    while True:
        try:
            p = {"prompt": prompt, "client_id": client_id}
            data = json.dumps(p).encode('utf-8')
            req = urllib.request.Request("http://{}/prompt".format(server_address), data=data)
            return json.loads(urllib.request.urlopen(req, timeout=5).read())
        except URLError:
            print("连接超时，正在重试...")

def get_history(prompt_id):
    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:
        return json.loads(response.read())

def get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:
        return response.read()

def handle_whitespace(string: str):
    return string.strip().replace("\n", " ").replace("\r", " ").replace("\t", " ")

def parse_name(ckpt_name):
    path = ckpt_name
    filename = path.split("/")[-1]
    filename = filename.split(".")[:-1]
    filename = ".".join(filename)
    return filename


def calculate_sha256(file_path):
    sha256_hash = hashlib.sha256()

    with open(file_path, "rb") as f:
        # Read the file in chunks to avoid loading the entire file into memory
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)

    return sha256_hash.hexdigest()

def get_images(ws, prompt):
    prompt_id = queue_prompt(prompt)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(prompt_id)[prompt_id]
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
                output_images[node_id] = images_output

    return output_images
    
def saveImages(result, imageDir,imageName,comment,previewImage=0): 
    metadata = PngInfo()
    metadata.add_text("parameters", comment)  
    k = 0 
    for node_id in result:    
        if k < previewImage:
            k += 1
            continue    
        for image_data in result[node_id]:            
            image = Image.open(io.BytesIO(image_data))
            imagePath = imageDir + "/" + imageName + f"_{k}" + ".png"
            image.save(imagePath,pnginfo=metadata) 
            k += 1
    

In [3]:
server_address = "127.0.0.1:8188"
client_id = str(uuid.uuid4())


## import json file

In [4]:
promptJson = 'api.json'

with open(promptJson,encoding="utf-8") as f:
    prompt = json.load(f)

In [5]:

# 原图文件夹
originalDir = f"c:/Users/BigHippo78/Pictures/cosplayer"

# 设置图片保存路径
imageDir = f"c:/Users/BigHippo78/Pictures/upscale_cosplayer"
# 如果文件夹不存在，创建文件夹
if not os.path.exists(imageDir):
    os.mkdir(imageDir)

In [6]:

from PIL import Image
from sdparsers import ParserManager
import websocket

ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))

batch_size = 1

# 遍历参考图片
for originalImag in os.listdir(originalDir):
    # 如果不是jpg或png文件，跳过
    if not (originalImag.endswith(".jpg") or originalImag.endswith(".png") or originalImag.endswith(".jpeg")):
        continue
    
    originalImagePath = originalDir + "/" + originalImag
    # 如果文件名包含_H或者_B，跳过
    if originalImag.find("_H") != -1 or originalImag.find("_B") != -1:
        print(f"{originalImag}包含_H或者_B,判定为mask文件，跳过")
        continue
    
    # 读取原图片名字，去掉后缀.png
    originalImagName = originalImag.split(".")[0]
    # 检测目标文件夹里是否已经存在该图片，如果存在，跳过
    if checkImageExsit(imageDir, originalImagName):
        print(f"{originalImagName}已经存在，跳过")
        continue  
    
    # 读取原图，以及其大小,然后控制生成图片的大小在1-1.2M之间
    with Image.open(originalImagePath) as originalImage:
        originalImageWidth = originalImage.size[0]
        originalImageHeight = originalImage.size[1]
    
    
    # 设置图片生成批次数量，以及原图路径       
    prompt['5']['inputs']['image'] = originalDir + "/" + originalImag
    
    comment = ''
    
    # 连接api，生成并保存图片
    start_time = time.time()    
    images = get_images(ws, prompt)   
    saveImages(images, imageDir, originalImagName, comment=comment,previewImage=0)     
    end_time = time.time()
    print(f"耗时{end_time-start_time}秒")
            

耗时1.0480473041534424秒
耗时0.8974688053131104秒
耗时0.8946874141693115秒
耗时0.9087276458740234秒
耗时0.9140450954437256秒
耗时0.9240200519561768秒
耗时0.934340238571167秒
耗时0.8469440937042236秒
耗时0.8771178722381592秒
耗时0.8336472511291504秒
耗时0.9299502372741699秒
耗时0.7261509895324707秒
耗时0.7204549312591553秒
耗时0.7233767509460449秒
耗时0.8284761905670166秒
耗时0.952770471572876秒
耗时0.864281177520752秒
耗时0.9030740261077881秒
耗时1.120431900024414秒
耗时0.9009826183319092秒
耗时0.8635585308074951秒
耗时0.9141833782196045秒
耗时0.8108806610107422秒
耗时0.8580031394958496秒
耗时1.1075172424316406秒
耗时0.9330580234527588秒
耗时0.9682319164276123秒
耗时0.9798247814178467秒
耗时0.8740432262420654秒
耗时0.8857529163360596秒
耗时0.8520019054412842秒
耗时0.8517327308654785秒
耗时0.8606112003326416秒
耗时0.9817399978637695秒
耗时0.8788652420043945秒
耗时0.9915690422058105秒
耗时0.9200222492218018秒
耗时0.8489696979522705秒
耗时0.8474769592285156秒
耗时0.8568699359893799秒
耗时0.949549674987793秒
耗时0.9191455841064453秒
耗时1.0858571529388428秒
耗时1.010434865951538秒
耗时1.045447826385498秒
耗时1.0535373687744